Aqui eu vou escrever todo o programa e explicar cada parte :)

Primeiro, vou apenas copiar os blocos de código do projeto do Matheus e escrever o que precisa ser alterado.

Uma coisa que tenho percebido ao redor do código é a falta de funções auxiliares. Eu pessoalmente não gosto de main longa e cheia, então acho que vou ajustar isso depois.

# 1. Crawler:

Aqui serão extraídos os nomes e _abstracts_ dos artigos relacionados aos termos de busca.

In [ ]:
import string, os

from Bio import Entrez
from pathlib import Path
import pubchempy as pcp

Essas são funções auxiliares para o funcionamento do _crawler_.

In [ ]:
def list_from_txt(file_path):
    strings_list = []
    with open(file_path, 'rt', encoding='utf-8') as list_file:
        for line in list_file: strings_list.append(line.rstrip('\n'))
    return strings_list

def search(paper_query):
    final_query = '{} AND English[Language]'.format(paper_query)
    Entrez.email = 'tirs@estudante.ufscar.br'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax='999999',
                            retmode='xml',
                            term=final_query)
    found = Entrez.read(handle)
    handle.close()
    return found

def fetch_details(paper_ids):
    ids_string = ','.join(paper_ids)
    Entrez.email = 'tirs@estudante.ufscar.br'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids_string)
    found = Entrez.read(handle)
    handle.close()
    return found

def flat(round_list):
    flat_list = []
    for element in round_list:
        if type(element) is list:
            for item in element: flat_list.append(item)
        else: flat_list.append(element)
    return flat_list

Essa ultima função auxiliar é a que insere aos termos de busca sinônimos de compostos para aumentar o escopo da busca.

ALTERAÇÕES NECESSÁRIAS: aqui está _hardcoded_ os IDs dos compostos que terão seus sinônimos inclusos. Esses compostos são tratamentos conhecidos para a AML, e foram levantados manualmente por um especialista.

SOLUÇÕES?: preciso substituir esses compostos manualmente escolhidos, arrumando alguma forma de obter esses IDs automaticamente para uma doença escolhida. Como? Não sei ainda.

In [ ]:
def extend_search():
    compound_ids = [
        14888, 9444, 62770, 2907, 6253, 122640033, 5743, 90480031, 76970819, 636362, 71657455, 9829523, 51082, 5865,
        2723601, 249332, 49846579, 11422859
    ]

    substance_ids = [
        404336834
    ]

    extended = [pcp.Compound.from_cid(compound_ids[0]).synonyms]

    for c in compound_ids[1:]:
        extended.extend(pcp.Compound.from_cid(c).synonyms)

    for s in substance_ids:
        extended.extend(pcp.Substance.from_sid(s).synonyms)

    extended = flat(extended)
    remove_words = []
    for s in extended:
        if '[as the base]' in s or '[Poison]' in s or '[ISO]' in s or '[INN]' in s or 'USP/JAN' in s or '(JAN' in s or '[JAN' in s or 'Latin' in s or 'Spanish' in s or '(TN)' in s or '(INN)' in s or 'USAN' in s or 'JP17/USP' in s or 'Czech' in s or 'German' in s or '[CAS]' in s:
            remove_words.append(s)

    extended = [x for x in extended if x not in remove_words]
    extended = [x for x in extended if len(x) >= 3]
    extended = list(dict.fromkeys(extended))

    useless_strings = list_from_txt('./data/bad_search_strings.txt')
    for w in useless_strings:
        try:
            extended.remove(w)
        except ValueError:
            pass

    remove_words = ['Antibiotic U18496', 'D,L-Cyclophosphamide', 'N,O-propylen-phosphorescence-ester-diamid',
                    'UNII-6UXW23996M component CMSMOCZEIVJLDB-CQSZACIVSA-N']
    for w in remove_words:
        try:
            extended.remove(w)
        except ValueError:
            pass

    return extended

Aqui é o programa principal do _crawler_. Rodar isso aqui demora muito, então não sei ainda como vou testar.

ALTERAÇÕES NECESSÁRIAS:

   1. Termos de busca não são fixos, preciso de uma função auxiliar que retorna termos de busca para a doença de escolha.

   2. Parecido com o 1, mas para os sinônimos.

   3. Não acho que esse arquivo de IDs vai continuar existindo.

In [ ]:
if __name__ == '__main__':
    destination_path = './results/'
    Path('./data/ids.txt').touch(exist_ok=True)
    Path(destination_path).mkdir(parents=True, exist_ok=True)

    # Cria uma lista com todos os termos de busca do arquivo. (não vai mais existir esse arquivo)
    queries = list_from_txt('./data/queries.txt')
    paper_counter = 0

    # Insere os sinônimos dos compostos no fim da lista.
    synonyms = extend_search()
    queries.extend(synonyms)

    # Cria uma lista com os IDs de todos os artigos já obtidos e um conjunto de IDs de artigos obtidos.
    old_papers = list_from_txt('./data/ids.txt')
    ids = set(old_papers)

    # Para cada termo de busca...
    for query in queries[100:]:
        # Normaliza o termo de busca.
        query = query.encode('ascii', 'ignore').decode('ascii')
        print('searching for {}'.format(query))

        # Procura pelo termo de busca, salva os IDs dos artigos encontrados em id_list.
        id_list = list(search(query)['IdList'])
        # Mantém só os que ainda não estão no conjunto de IDs conhecidos, ou seja, os novos.
        id_list = [x for x in id_list if x not in ids]

        # Se não achar nada novo, pode ir para o próximo termo de busca.
        if not id_list:
            print('No new papers found\n')
            continue

        print('{} papers found\n'.format(len(id_list)))
        # Insere os novos IDs no conjunto.
        ids.update(id_list)

        # Pega os detalhes de cada artigo novo.
        papers = fetch_details(id_list)

        # Cria uma pasta com o nome do termo de busca, formatado para poder ser nome de pasta.
        folder_name = query.lower().translate(str.maketrans('', '', string.punctuation)).replace(' ', '_')
        Path(destination_path + '{}'.format(folder_name)).mkdir(parents=True, exist_ok=True)

        # Para cada artigo novo...
        for paper in papers['PubmedArticle']:
            article_year = ''
            article_abstract = ''

            # Se não tiver título, vai para o próximo
            try:
                article_title = paper['MedlineCitation']['Article']['ArticleTitle']
                article_title_filename = article_title.lower().translate(
                    str.maketrans('', '', string.punctuation)).replace(' ', '_')
            except KeyError as e: continue

            # Se não tiver prefácio, vai para o próximo.
            try:
                article_abstract = ' '.join(paper['MedlineCitation']['Article']['Abstract']['AbstractText'])
                article_year = paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year']
            except KeyError as e:
                if 'Abstract' in e.args: continue
                if 'Year' in e.args:
                    article_year = paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate'][
                                       'MedlineDate'][0:4]

            # Se não tiver ano válido, vai para o próximo.
            if len(article_year) != 4: continue

            # O nome do arquivo vai ser "{ano}_{nome_do_artigo}"...
            filename = '{}_{}'.format(article_year, article_title_filename)
            if len(filename) > 150: filename = filename[0:146]

            # e vai ser escrito naquela pasta criada antes do loop.
            path_name = destination_path + folder_name + '/{}.txt'.format(filename)
            path_name = path_name.encode('ascii', 'ignore').decode('ascii')

            # Escreve o arquivo.
            with open(path_name, "a", encoding='utf-8') as file:
                file.write(article_title + ' ' + article_abstract)

            paper_counter += 1

        # Coloca os IDs dos artigos novos no arquivo.
        with open('./data/ids.txt', 'a+', encoding='utf-8') as file:
            for new_id in id_list: file.write('\n' + str(new_id))

    print('Crawler finished with {} papers collected.'.format(len(old_papers) + paper_counter))

# 2. Agregação dos textos por ano

Após pegar todos os prefácios resultantes das buscas, os textos precisam ser agrupados por ano para realizar aquela análise de quão antes o tratamento foi descoberto.

In [ ]:
if __name__ == '__main__':
    start_year = 1900
    end_year = 2025
    source_path = './results/'
    destination_path = './results_aggregated/'

    # Pega os nomes de todos os arquivos que vieram do crawler.
    filenames = list(map(str, Path(source_path).glob('**/*.txt')))
    os.makedirs(destination_path, exist_ok=True)

    # Para cada ano no intervalo...
    for year in range(start_year, end_year + 1):
        # Pega os nomes dos arquivos de artigos que são do ano atual.
        filenames_current_year = [f for f in filenames if Path(f).stem.startswith(str(year))]

        # Se não tiver nenhum desse ano, vai para o próximo.
        if not filenames_current_year:
            continue

        print(f'{len(filenames_current_year)} papers from {year}.\n')

        # Pega os prefácios de todos os artigos desse ano.
        abstracts = []
        for fname in filenames:
            with open(fname, encoding='utf-8') as infile:
                abstracts.extend(infile.readlines())

        # Junta tudo em um só arquivo texto.
        output_file = Path(destination_path) / f'results_file_1900_{year}.txt'
        with open(output_file, 'w', encoding='utf-8') as f:
            for fname, abstract in zip(filenames, abstracts):
                f.write(f"{Path(fname).stem[5:]}|{abstract}")

# 3. Pré-processamento dos textos

Aqui os textos serão traduzidos em DataFrames para melhorar o desempenho do processamento.

In [ ]:
import pyspark.sql.functions as F
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.window import Window
import pyspark.sql.types as T

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from functools import reduce

As funções auxiliares são altamente customizadas para a LMA, muitas coisas vão ter que mudar.

In [ ]:
def ss():
    sc = SparkContext.getOrCreate()
    return SparkSession(sc)

def dataframes_from_txt(summaries_path):
    filenames = sorted([str(x) for x in Path(summaries_path).glob('*.txt')])
    dataframes = []

    # Para cada arquivo no diretório...
    for file_path in filenames:
        # Pega o ano.
        year_of_file = file_path\
            .replace(os.path.join(summaries_path, 'results_file_1900_'), '')\
            .replace('.txt', '')

        NATURE_FILTERED_WORDS_IN_TITLE = [
            'foreword', 'prelude', 'commentary', 'workshop', 'conference', 'symposium',
            'comment', 'retract', 'correction', 'erratum', 'memorial'
        ]

        # Filtra os artigos com título contendo pelo menos uma dessas palavras.
        title_doesnt_have_nature_filtered_words = reduce(
            lambda acc, word: acc & (F.locate(word, F.col('title')) == F.lit(0)),
            NATURE_FILTERED_WORDS_IN_TITLE,
            F.lit(True)
        )

        # Cria uma tabela para cada arquivo com as colunas filename, title, summary e id.
        df = ss()\
            .read\
            .option('header', 'false')\
            .option('lineSep', '\n')\
            .option('sep', '|')\
            .option('quote', '')\
            .csv(file_path)\
            .withColumn('filename', F.lit(year_of_file))\
            .withColumnRenamed('_c0', 'title')\
            .withColumnRenamed('_c1', 'summary')\
            .where(title_doesnt_have_nature_filtered_words)\
            .withColumn('id', F.monotonically_increasing_id())

        # Coloca essa tabela na lista de tabelas.
        dataframes.append(df)

    # Junta tudo em uma tabela só e retorna.
    return reduce(lambda df1, df2: df1.union(df2), dataframes)

def get_csv_in_folder(file_path):
    """Search for a .csv file in a given path. It must find just one .csv file - this constraint is tested with assert command.
    This is an auxiliar function used during reading .csv PySpark DataFrames.
    Args:
        file_path: path to the folder containg the .csv file.
    """

    files = os.listdir(file_path)
    files = filter(lambda x: x[-3:] == 'csv', files)
    files = list(files)

    assert len(files) == 1, files

    return os.path.join(file_path, files[0])

def read_csv_table_files(file_path, sep=','):
    full_path = file_path

    if file_path[-3:] != 'csv':
        file_path = get_csv_in_folder(file_path)

    return ss()\
        .read\
        .option('header', 'true')\
        .option('sep', sep)\
        .csv(full_path)

def to_csv(df, target_folder, num_files=1, sep=','):
    """Saves a PySpark Dataframe into .csv file.
    Args:
        df: object of the DataFrame;
        target_folder: path where the .csv is going to be saved;
        num_files: number of .csv files to be created, default is 1.
    """

    return df\
        .coalesce(num_files)\
        .write\
        .mode('overwrite')\
        .option('header', 'true')\
        .option('sep', sep)\
        .format('csv')\
        .save(target_folder)

def get_wordnet_pos(treebank_tag):
    """Returns WORDNET POS compliance to WORDNET lemmatization (ADJ, VERB, NOUN, ADV)"""

    if treebank_tag.startswith('J'):
            return 'a'
    elif treebank_tag.startswith('V'):
            return 'v'
    elif treebank_tag.startswith('N'):
            return 'n'
    elif treebank_tag.startswith('R'):
            return 'r'
    else:
    # As default pos in lemmatization is Noun
        return 'n'

def tokenize(data):
    """Toeknizes a sentence

        Args:
        data: a sentence (string).
    """
    if data is None:
        return ['']

    else:
        return word_tokenize(data)

def remove_last_digit(data):
    """Removes the last character of a string a sentence

        Args:
        data: a string.
    """

    return data[:-1]

def return_last_digit(data):
    """Toeknizes a sentence

        Args:
        data: a sentence (string).
    """

    try:
        aux = data[-1]

    except:
        aux = ''

    return aux

def summary_column_preprocessing(column, bert_model=False):
    """Executes intial preprocessing in a PySpark text column. It removes some unwanted regex from the text.
    Args:
        column: the name of the column to be processed.
    """

    aml_synonyms = [
        'acute[- ]?myeloid[- ]?leukemia',
        'acute myelocytic leukemia',
        'acute myelogenous leukemia',
        'acute granulocytic leukemia',
        'acute non-lymphocytic leukemia',
        'acute mylogenous leukemia',
        'acute myeloid leukemia',
        'acute nonlymphoblastic leukemia',
        'acute myeloblastic leukemia',

        # subtipos de AML:
        'acute erythroid leukemia',
        'acute myelomonocytic leukemia',
        'acute monocytic leukemia',
        'acute megakaryoblastic leukemia',
        'acute promyelocytic leukemia',
    ]

    regex = r'(?i)({})'.format('|'.join(aml_synonyms))

    column = F.trim(column)

    column = F.regexp_replace(column, r'<[^>]+>', '')
    column = F.regexp_replace(column, r'([--:\w?@%&+~#=]*\.[a-z]{2,4}\/{0,2})((?:[?&](?:\w+)=(?:\w+))+|[--:\w?@%&+~#=]+)?', '')

    if bert_model == False:
        column = F.regexp_replace(column, r'[;:\(\)\[\]\{\}.,"!#$&\'*?@\\\^`|~]', '')

    column = F.regexp_replace(column, r'\s+', ' ')
    column = F.regexp_replace(column, r'(?i)(leukaemia)', 'leukemia')
    column = F.regexp_replace(column, r'(?i)(leukaemic)', 'leukemic')

    column = F.regexp_replace(column, regex, 'AML')
    column = F.regexp_replace(column, regex, 'AML')

    column = F.regexp_replace(column, r'(?i)(acute myeloid or (lymphoblastic|lymphoid) leukemia[s]?)', 'AML or ALL')
    column = F.regexp_replace(column, r'(?i)(acute myeloid and (lymphoblastic|lymphoid) leukemia[s]?)', 'AML and ALL')
    column = F.regexp_replace(column, r'(?i)(acute myeloid and chronic lymphocytic leukemia[s]?)', 'AML and CLL')

    # cytarabine
    column = F.regexp_replace(column, r'(?i)( cytosine arabinoside triphosphate )', ' cytarabinetriphosphate ')
    column = F.regexp_replace(column, r'(?i)(cytosine arabinoside triphosphate )', 'cytarabinetriphosphate ')
    column = F.regexp_replace(column, r'(?i)(\(cytosine arabinoside triphosphate\))', '(cytarabinetriphosphate)')
    column = F.regexp_replace(column, r'(?i)( cytosine arabinoside triphosphate.)', ' cytarabinetriphosphate.')
    column = F.regexp_replace(column, r'(?i)( cytosine arabinoside triphosphate,)', ' cytarabinetriphosphate,')
    column = F.regexp_replace(column, r'(?i)( cytosine arabinoside monophosphate )', ' cytarabinemonophosphate')
    column = F.regexp_replace(column, r'(?i)( cytosine arabinoside monophosphate.)', ' cytarabinemonophosphate.')
    column = F.regexp_replace(column, r'(?i)( cytosine arabinoside monophosphate,)', ' cytarabinemonophosphate,')
    column = F.regexp_replace(column, r" arabinocytidine 5' phosphate|arabinofuranosylcytosine 5'-triphosphate|Ara-CTP", " cytarabine5phosphate")
    column = F.regexp_replace(column, r"(?i)(1-beta-D|1-β-d|1β-d|1 beta-D)-Arabinofuranosylcytosine 5'-triphosphate", "cytarabine5phosphate")
    column = F.regexp_replace(column, r'(?i)( cytosine arabinoside | \[Ara-C\] )', ' cytarabine ')
    column = F.regexp_replace(column, r'(?i)( cytosine arabinoside\.)', ' cytarabine.')
    column = F.regexp_replace(column, r'(?i)( cytosine arabinoside,)', ' cytarabine,')
    column = F.regexp_replace(column, r'(?i)( cytosine arabinoside:)', ' cytarabine:')
    column = F.regexp_replace(column, r'(?i)(\(cytosine arabinoside\))', '(cytarabine)')
    column = F.regexp_replace(column, r'(?i)(\(ara[-]?c\))', '(cytarabine)')
    column = F.regexp_replace(column, r'(?i)(\(ara[-]?c, )', '(cytarabine, ')
    column = F.regexp_replace(column, r'(?i)(\(ara[-]?c )', '(cytarabine ')
    column = F.regexp_replace(column, r'(?i)( ara[-]?c\))', ' cytarabine)')
    column = F.regexp_replace(column, r'(?i)( ara[-]?c/)', ' cytarabine/)')
    column = F.regexp_replace(column, r'(?i)(/ara[-]?c\.)', '/cytarabine.')
    column = F.regexp_replace(column, r'(?i)(/ara[-]?c )', '/cytarabine ')
    column = F.regexp_replace(column, r'(?i)(ara[-]?c-induced)', 'cytarabine-induced')
    column = F.regexp_replace(column, r'(?i)(ara[-]?c-treated)', 'cytarabine-treated')
    column = F.regexp_replace(column, r'(?i)(ara[-]?c-based)', 'cytarabine-based')
    column = F.regexp_replace(column, r'(?i)(ara[-]?c-resistant)', 'cytarabine-resistant')
    column = F.regexp_replace(column, r'(?i)(/ara[-]?c/)', '/cytarabine/')
    column = F.regexp_replace(column, r'(HDAra-C|HD-Ara-C)', 'high-dose cytarabine')
    column = F.regexp_replace(column, r'(LDAra-C|LD-Ara-C|LDAC)', 'low-dose cytarabine')
    column = F.regexp_replace(column, r'(IDAra-C|ID-Ara-C)', 'intermediate-dose cytarabine')
    column = F.regexp_replace(column, r'\(Ara-C;', '(cytarabine;')
    column = F.regexp_replace(column, r'\[3H\]Ara-C', '([3H]cytarabine')
    column = F.regexp_replace(column, r' Arabinocytidine', ' Cytarabine')
    column = F.regexp_replace(column, r"(?i)(1-beta-D|1-β-d|1β-d|1 beta-D)-Arabinofuranosyl[-]?cytosine ", "cytarabine ")
    column = F.regexp_replace(column, r"(?i)(1-beta-D|1-β-d|1β-d|1 beta-D)-Arabinofuranosyl[-]?cytosine,", "cytarabine,")
    column = F.regexp_replace(column, r"(?i)(1-beta-D|1-β-d|1β-d|1 beta-D)-Arabinofuranosyl[-]?cytosine.", "cytarabine.")
    column = F.regexp_replace(column, r'(?i)( Arabinofuranosylcytosine | Ara-C )', " cytarabine ")
    column = F.regexp_replace(column, r'(?i)( Arabinofuranosylcytosine,)', " cytarabine,")
    column = F.regexp_replace(column, r'(?i)( Arabinofuranosylcytosine\.)', " cytarabine.")
    column = F.regexp_replace(column, r' arabinocytidine ', ' cytarabine ')
    column = F.regexp_replace(column, r' arabinocytidine,', ' cytarabine,')
    column = F.regexp_replace(column, r' arabinocytidine:', ' cytarabine:')
    column = F.regexp_replace(column, r' arabinocytidine\.', ' cytarabine.')
    column = F.regexp_replace(column, r'147-94-4', 'cytarabine')
    column = F.regexp_replace(column, r'\(Cytosar-U\)|\(Cytosar\)', '(cytarabine)')
    column = F.regexp_replace(column, r'(?i)\(Cytosar-U,', '(cytarabine,')
    column = F.regexp_replace(column, r'(?i) cytosar\)', ' cytarabine)')
    column = F.regexp_replace(column, r' aracytin[e]? ', ' cytarabine ')
    column = F.regexp_replace(column, r' aracytin[e]?\.', ' cytarabine.')
    column = F.regexp_replace(column, r' aracytin[e]?,', ' cytarabine,')
    column = F.regexp_replace(column, r'\+aracytine\+', '+cytarabine+')
    column = F.regexp_replace(column, r'-aracytine ', '-cytarabine ')
    column = F.regexp_replace(column, r'-aracytine\.', '-cytarabine.')
    column = F.regexp_replace(column, r'-aracytine:', '-cytarabine:')
    column = F.regexp_replace(column, r'(?i)(Cytosine beta-D-arabinoside)', 'cytarabine')
    column = F.regexp_replace(column, r'(?i)(Cytosine-1-beta-D-arabinofuranoside)', 'cytarabine')
    column = F.regexp_replace(column, r'(?i)(beta-ara c)', 'cytarabine')
    column = F.regexp_replace(column, r'(?i)(liposomal cytarabine)', 'liposomalcytarabine')
    column = F.regexp_replace(column, r'(?i)(depocyte)', 'liposomalcytarabine')
    column = F.regexp_replace(column, r"4'-thio-ara-C", "4-thio-cytarabine")
    column = F.regexp_replace(column, r"NSC[ -]63878", "cytarabine")
    column = F.regexp_replace(column, r"ofcytarabine", "of cytarabine")

    # daunorubicin:
    column = F.regexp_replace(column, r'[Dd]aunomycin \(DAU\)|[Dd]aunorubicin \(DAU\)|daunorubicin hydrochloride \(DAU\)', 'daunorubicin (daunorubicin)')
    column = F.regexp_replace(column, r'NSC[ -]?82151|Rubomycin C|[^R]DNX[^B]', 'daunorubicin')
    column = F.regexp_replace(column, r'daunomycin-', 'daunorubicin-')
    column = F.regexp_replace(column, r'Daunomycin-', 'Daunorubicin-')
    column = F.regexp_replace(column, r'(?i)( daunorubicine )', ' daunorubicin ')
    column = F.regexp_replace(column, r'\(DNR\)', '(daunorubicin)')
    column = F.regexp_replace(column, r'\(daunomycin ', '(daunorubicin ')
    column = F.regexp_replace(column, r'rubidomicine|rubidomicin', 'daunorubicin')
    column = F.regexp_replace(column, r'leukaemomycin C', 'daunorubicin')
    column = F.regexp_replace(column, r'(Cerubidine)', '(daunorubicin)')
    column = F.regexp_replace(column, r'[Ll]iposomal daunorubicin', 'liposomaldaunorubicin')
    column = F.regexp_replace(column, r'(?i)daunoxome', 'daunorubicin')
    column = F.regexp_replace(column, r'LDL-daunomycin|LDL:daunomycin', 'low-density-lipoproteins-daunorubicin')
    column = F.regexp_replace(column, r'\(daunomycin\)', '(daunorubicin)')
    column = F.regexp_replace(column, r'\(daunomycin,', '(daunorubicin,')
    column = F.regexp_replace(column, r'LDL-daunomycin', 'low-density-lipoproteins-daunorubicin')
    column = F.regexp_replace(column, r'[\[]?3H[\]]?[-]?daunomycin', '3H-daunorubicin')
    column = F.regexp_replace(column, r' daunomycin\.', ' daunorubicin.')
    column = F.regexp_replace(column, r'13-dihydrodaunomycin', '13-dihydrodaunorubicin')
    column = F.regexp_replace(column, r'cys-aconytil-daunomycin', 'cys-aconytil-daunorubicin')
    column = F.regexp_replace(column, r'\[daunomycin\]', '[daunorubicin]')
    column = F.regexp_replace(column, r'porphyrin-daunomycin[a-z]', 'Por-(daunorubicin)')

    # azacitidine:
    column = F.regexp_replace(column, r'5-AZA|5-azaC|5-AZAC|5-Aza|5-ACR|5-AC', 'azacitidine')
    column = F.regexp_replace(column, r" 5[']?[-]?azac[yi]tidine ", ' azacitidine ')
    column = F.regexp_replace(column, r' 5-azac[yi]tidine-', ' azacitidine-')
    column = F.regexp_replace(column, r' 5-azac[yi]tidine\.', ' azacitidine.')
    column = F.regexp_replace(column, r' 5-azac[yi]tidine;', ' azacitidine:')
    column = F.regexp_replace(column, r' 5-azac[yi]tidine,', ' azacitidine,')
    column = F.regexp_replace(column, r' 5-azac[yi]tidine\)', ' azacitidine)')
    column = F.regexp_replace(column, r'(?i)(5-aza-CR)', 'azacitidine')
    column = F.regexp_replace(column, r'(?i)( azac[yi]tidine )', ' azacitidine ')
    column = F.regexp_replace(column, r'\(AZA\)', '(azacitidine)')
    column = F.regexp_replace(column, r'(?i)(vidaza)', 'azacitidine')

    # gemtuzumab-ozogamicin:
    column = F.regexp_replace(column, r"(?i)(gemtuzumab[- ]?ozogam[yi]cin) \(GO\)", 'gemtuzumab-ozogamicin (gemtuzumab-ozogamicin)')
    column = F.regexp_replace(column, r"(?i)(gemtuzumab[- ]?ozogam[yi]cin) \(GO,", 'gemtuzumab-ozogamicin (gemtuzumab-ozogamicin,')
    column = F.regexp_replace(column, r"(?i)(gemtuzumab[- ]?ozogam[yi]cin) \(GO;", 'gemtuzumab-ozogamicin (gemtuzumab-ozogamicin;')
    column = F.regexp_replace(column, r"(?i)(gemtuzumab ozogam[yi]cin)", 'gemtuzumab-ozogamicin')
    column = F.regexp_replace(column, r"CMA-676|FLASI-GO", 'gemtuzumab-ozogamicin')
    column = F.regexp_replace(column, r"(?i)(my[o]?lotarg)", 'gemtuzumab-ozogamicin')

    # midostaurin:
    column = F.regexp_replace(column, r"(?i)(4'-N-benzoyl staurosporine|4'-N-Benzoylstaurosporine|N-Benzoylstaurosporine)", 'midostaurin')
    column = F.regexp_replace(column, r"(?i)(rydapt)", 'midostaurin')
    column = F.regexp_replace(column, r"PKC[ -]?412|CGP[ -]?41251|CAS 120685-11-2", 'midostaurin')

    # CPX-351 (ou vyxeos):
    column = F.regexp_replace(column, r"(?i)(cpx[- ]?351)", 'vyxeos')
    column = F.regexp_replace(column, r"(?i)(vyxeos liposomal)", 'vyxeos')
    column = F.regexp_replace(column, r"(?i)(Daunorubicin[ ]?\/[ ]?cytarabine liposome|liposomaldaunorubicin/cytarabine)", 'vyxeos')

    # ivosidenib:
    column = F.regexp_replace(column, r"(?i)(tibsovo)", 'ivosidenib')
    column = F.regexp_replace(column, r"(?i)(ag120)", 'ivosidenib')

    # venetoclax:
    column = F.regexp_replace(column, r"(?i)(ABT[ -]?199|GDC[ -]?0199|venclyxto|venclexta)", 'venetoclax')

    # enasidenib:
    column = F.regexp_replace(column, r"(?i)(ag[ -]?221|idhifa|cc[ -]?90007)", 'enasidenib')

    # gilteritinib:
    column = F.regexp_replace(column, r"Xospata|ASP2215", 'gilteritinib')

    # glasdegib:
    column = F.regexp_replace(column, r"\(DAU\)", '(glasdegib)')
    column = F.regexp_replace(column, r"(DAURISMO|PF[ -][0]?4449913|PF[ -]913)", 'glasdegib')

    # arsenic trioxide:
    column = F.regexp_replace(column, r"(?i)(arsenic trioxide)", 'arsenictrioxide')
    column = F.regexp_replace(column, r"As2[O0]3,", 'arsenictrioxide,')
    column = F.regexp_replace(column, r"As2[O0]3\)", 'arsenictrioxide)')
    column = F.regexp_replace(column, r"As2[O0]3\.", 'arsenictrioxide.')
    column = F.regexp_replace(column, r"As2[O0]3$", 'arsenictrioxide.')
    column = F.regexp_replace(column, r"As2[O0]3 ", 'arsenictrioxide ')
    column = F.regexp_replace(column, r"As2[O0]3", 'arsenictrioxide')
    column = F.regexp_replace(column, r"As\(2\)[O0]\(3\) ", 'arsenictrioxide ')
    column = F.regexp_replace(column, r" ATO ", ' arsenictrioxide ')
    column = F.regexp_replace(column, r"\(ATO\)", '(arsenictrioxide)')
    column = F.regexp_replace(column, r"\(ATO[,;]", '(arsenictrioxide,')
    column = F.regexp_replace(column, r" ATO-", ' arsenictrioxide-')
    column = F.regexp_replace(column, r"(?i)(trisenoxt|trisenox)", 'arsenictrioxide')
    column = F.regexp_replace(column, r"(?i)(arsenic\(III\) oxide)", 'arsenictrioxide')

    # cyclophosphamide:
    column = F.regexp_replace(column, r"(?i)(methylerythritol cyclophosphane)", 'methylerythritolcyclophosphamide')
    column = F.regexp_replace(column, r"(?i)(cyclophosphane)", 'cyclophosphamide')
    column = F.regexp_replace(column, r"cyclophosphamide-endoxan|Cyklofosfamid|NSC[ -]?26271|Genoxal", 'cyclophosphamide')
    column = F.regexp_replace(column, r"(?i)(endoxane|endoxan|cyclophosphamidum)", 'cyclophosphamide')
    column = F.regexp_replace(column, r"(?i)( neosar\))", ' cyclophosphamide)')

    # dexamethasone:
    column = F.regexp_replace(column, r"(?i)(decadron|dexasone|dexason|Dextenza|Dexycu)", 'dexamethasone')
    column = F.regexp_replace(column, r"Maxidex|dexasone|Hexadrol|Oradexon|Fortecortin", 'dexamethasone')
    column = F.regexp_replace(column, r"[Ii]ntratympanic dexamethazone", 'intratympanicdexamethasone')
    column = F.regexp_replace(column, r"(?i)(de[sx]ametha[sz]one|Desametasone)", 'dexamethasone')
    column = F.regexp_replace(column, r"dexamethasone \(DMS\)", 'dexamethasone (dexamethasone)')

    # idarubicin:
    column = F.regexp_replace(column, r"NSC-256439|Zavedos", 'idarubicin')
    column = F.regexp_replace(column, r"\(4-demethoxydauno(mycin|rubicin)\)", '(idarubicin)')
    column = F.regexp_replace(column, r"\(4-demethoxydaunorubicin;", '(idarubicin;')
    column = F.regexp_replace(column, r" 4-demethoxydauno(mycin|rubicin) ", ' idarubicin ')
    column = F.regexp_replace(column, r" 4-demethoxydauno(mycin|rubicin)\.", ' idarubicin.')
    column = F.regexp_replace(column, r" 4-demethoxydauno(mycin|rubicin),", ' idarubicin,')
    column = F.regexp_replace(column, r"\[14-14C\]4-demethoxydaunorubicin HCl", '[14-14C]idarubicinhcl')
    column = F.regexp_replace(column, r"(?i)(Idarubicine)", 'idarubicin')
    column = F.regexp_replace(column, r"(?i)(idarubicin hcl)", 'idarubicin')
    column = F.regexp_replace(column, r"(?i)(4-DMD[N]?R )", 'idarubicin ')
    column = F.regexp_replace(column, r"(?i)(4-DMD[N]?R[.;])", 'idarubicin,')
    column = F.regexp_replace(column, r"(?i)(4-DMD[N]?R\.)", 'idarubicin.')
    column = F.regexp_replace(column, r"(?i)(4-DMD[N]?R\))", 'idarubicin)')
    column = F.regexp_replace(column, r"(?i)(4-DMD[N]?R-)", 'idarubicin')

    # mitoxantrone:
    column = F.regexp_replace(column, r"mitoxantron,cytarabine", 'mitoxantrone, cytarabine')
    column = F.regexp_replace(column, r"(?i)(mito[zx]ant[h]?rone|novantrone)", 'mitoxantrone')
    column = F.regexp_replace(column, r"NSC[ -]?301739", 'mitoxantrone')
    column = F.regexp_replace(column, r"CL[ -]232[,]?315", 'mitoxantronehydrochloride')
    column = F.regexp_replace(column, r"mitoxantron ", 'mitoxantrone ')
    column = F.regexp_replace(column, r"mitoxantron\.", 'mitoxantrone.')
    column = F.regexp_replace(column, r"(?i)(mitoxantrone \(mit\))", 'mitoxantrone (mitoxantrone)')
    column = F.regexp_replace(column, r"(?i)(mitoxantrone \(mit,)", 'mitoxantrone (mitoxantrone,')
    column = F.regexp_replace(column, r"(?i)(mitoxantrone) \((MTX|MIP|MX)\)", 'mitoxantrone (mitoxantrone)')
    column = F.regexp_replace(column, r"(?i)(mitoxantrone hydrochloride) \((MIT|MTO)\)", 'mitoxantronehydrochloride (mitoxantronehydrochloride)')
    column = F.regexp_replace(column, r"(?i)(mitoxantrone hydrochloride)", 'mitoxantronehydrochloride')
    column = F.regexp_replace(column, r"(?i)(1,4-dihydroxy-5,8-bis[ ]?\(\([ ]?\(2-\[\(2-hydroxyethyl\)amino\]ethyl\)[ -]?amino\)\)-9,10-anthracenedione dihydrochloride)", 'mitoxantrone')

    # pemigatinib:
    column = F.regexp_replace(column, r"PEMAZYRE", 'pemigatinib')

    # prednisone:
    column = F.regexp_replace(column, r"1-dehydrocortisone|[Dd]eltasone|meticorten|NSC[ -]?10023", 'prednisone')
    column = F.regexp_replace(column, r"(?i)(ultracorten-H|ultracorten H|ultracortene|ultracorten)", 'prednisone')

    # rituximab:
    column = F.regexp_replace(column, r"Rituxan", 'Rituximab')
    column = F.regexp_replace(column, r"rituxan|\[RTX-(EU|US)\]|Truxima|CT-P10", 'rituximab')
    column = F.regexp_replace(column, r"(?i)(MabThera)", 'rituximab')
    column = F.regexp_replace(column, r"rituximab/Rituximab", 'rituximab rituximab')

    # thioguanine:
    column = F.regexp_replace(column, r"[Tt]ioguanine|NSC-752", 'thioguanine')
    column = F.regexp_replace(column, r"(?i)(6-thioguanine| 6 thioguanine)", 'thioguanine')
    column = F.regexp_replace(column, r"daunorubicin-cytarabine-6 thioguanine", 'daunorubicin cytarabine thioguanine')
    column = F.regexp_replace(column, r"\(6TG\)", '(thioguanine)')
    column = F.regexp_replace(column, r" 6TG ", ' thioguanine ')
    column = F.regexp_replace(column, r" 6TG\.", ' thioguanine.')
    column = F.regexp_replace(column, r" 6TG,", ' thioguanine,')
    column = F.regexp_replace(column, r"6TG-", 'thioguanine-')
    column = F.regexp_replace(column, r"2-amino-6-mercaptopurine|6-mercaptoguanine", 'thioguanine')
    column = F.regexp_replace(column, r"6[ -]?TG[Rr]", 'thioguanine resistance')
    column = F.regexp_replace(column, r"(?i)(thioguanine \(tg\))", 'thioguanine (thioguanine)')

    # vincristine:
    column = F.regexp_replace(column, r"vincrystine|NSC[ -]67574", 'vincristine')
    column = F.regexp_replace(column, r"(?i)(vincristine sulfate)", 'vincristinesulfate')
    column = F.regexp_replace(column, r"\[3H\][-]?vincristine|3Hvincristine", '3H-vincristine')
    column = F.regexp_replace(column, r"\[3H\][-]?VCR|3HVCR", '3H-vincristine')
    column = F.regexp_replace(column, r"[Vv]incristine \(VCR\)", 'vincristine (vincristine)')
    column = F.regexp_replace(column, r"CAS 57-22-7", 'vincristine')
    column = F.regexp_replace(column, r"vincristine,cytarabine", 'vincristine, cytarabine')

    # C-1027:
    column = F.regexp_replace(column, r"(?i)(lidamycin\(LDM\))", 'c-1027 (c-1027)')
    column = F.regexp_replace(column, r"lidamycin", 'c-1027')

    # glyceryl behenate:
    column = F.regexp_replace(column, r"glyceryl behenate", 'glycerylbehenate')
    column = F.regexp_replace(column, r"[Cc]ompritol 888 ATO", 'glycerylbehenate')

    # decitabine:
    column = F.regexp_replace(column, r"2'-deoxy-(beta-D|beta-d|β-d|β-D)-5-azacytidine", 'decitabine')
    column = F.regexp_replace(column, r"2'-deoxy-(beta-L|beta-l|β-l|β-L)-5-azacytidine", 'l-decitabine')

    # daunomycinone:
    column = F.regexp_replace(column, r'daunomycin aglycone', 'daunomycinone')
    column = F.regexp_replace(column, r'13-dihydrodaunomycinone', 'feudomycinonea')

    # valrubicin:
    column = F.regexp_replace(column, r'AD32|AD 32|AD-32', 'valrubicin')
    column = F.regexp_replace(column, r'(?i)(N-Trifluoroacetyladriamycin[ -]14-valerate)', 'valrubicin')

    # carmustine:
    column = F.regexp_replace(column, r'NSC-409962', 'carmustine')
    column = F.regexp_replace(column, r'BCNU|BCNU-NSC', 'carmustine')

    # dextromethorphan:
    column = F.regexp_replace(column, r'DXM[S]?', 'dextromethorphan')

    # docetaxel:
    column = F.regexp_replace(column, r'NSC[ -]?628503|RP[ -]?56976', 'docetaxel')

    # dactinomycin:
    column = F.regexp_replace(column, r'[Aa]ctinomycin[ -]D', 'dactinomycin')

    if bert_model == False:
        column = F.lower(column)

    return column

def words_preprocessing(df, column='word', bert_model=False):
    fix_typos_dict = {
        'citarabine': 'cytarabine',
	    'hdara-c': 'high-dose cytarabine',
	    'no-arac': 'n4-octadecyl-1-beta-d-arabinofuranosylcytosine',
	    'ara-c-ab': 'arac-agarose-bead',
	    'anhydro-ara-fc': "2,2'-anhydro-1-beta-d-arabinofuranosyl-5-fluorocytosine",
        'mol-ecule': 'molecule',
        '‑': '-',
        '‒': '-',
        '–': '-',
        '—': '-',
        '¯': '-',
        'à': 'a',
        'á': 'a',
        'â': 'a',
        'ã': 'a',
        'ä': 'a',
        'å': 'a',
        'ç': 'c',
        'è': 'e',
        'é': 'e',
        'ê': 'e',
        'ë': 'e',
        'í': 'i',
        'î': 'i',
        'ï': 'i',
        'ñ': 'n',
        'ò': 'o',
        'ó': 'o',
        'ô': 'o',
        'ö': 'o',
        '×': 'x',
        'ø': 'o',
        'ú': 'u',
        'ü': 'u',
        'č': 'c',
        'ğ': 'g',
        'ł': 'l',
        'ń': 'n',
        'ş': 's',
        'ŭ': 'u',
        'і': 'i',
        'ј': 'j',
        'а': 'a',
        'в': 'b',
        'н': 'h',
        'о': 'o',
        'р': 'p',
        'с': 'c',
        'т': 't',
        'ӧ': 'o',
        '⁰': '0',
        '⁴': '4',
        '⁵': '5',
        '⁶': '6',
        '⁷': '7',
        '⁸': '8',
        '⁹': '9',
        '₀': '0',
        '₁': '1',
        '₂': '2',
        '₃': '3',
        '₅': '5',
        '₇': '7',
        '₉': '9',
    }

    units_and_symbols = [
        '/μm', '/mol', '°c', '≥', '≤', '<', '>', '±', '%', '/mumol',
        'day', 'month', 'year', '·', 'week', 'days',
        'weeks', 'years', '/µl', 'μg', 'u/mg',
        'mg/m', 'g/m', 'mumol/kg', '/week', '/day', 'm²', '/kg', '®',
        'ﬀ', 'ﬃ', 'ﬁ', 'ﬂ', '£', '¥', '©', '«', '¬', '®', '°', '±', '²', '³',
        '´', '·', '¹', '»', '½', '¿',
         '׳', 'ᇞ​', '‘', '’', '“', '”', '•',  '˂', '˙', '˚', '˜' ,'…', '‰', '′',
        '″', '‴', '€',
        '™', 'ⅰ', '↑', '→', '↓', '∗', '∙', '∝', '∞', '∼', '≈', '≠', '≤', '≥', '≦', '≫', '⊘',
        '⊣', '⊿', '⋅', '═', '■', '▵', '⟶', '⩽', '⩾', '、', '气', '益', '粒', '肾', '补',
        '颗', '', '', '', '', '，'
    ]

    if bert_model:
        units_and_symbols_expr = '(%s)' % '|'.join(units_and_symbols[28:])

    else:
        units_and_symbols_expr = '(%s)' % '|'.join(units_and_symbols)

    def __keep_only_compound_numbers():
        return F.when(
            F.regexp_replace(F.lower(F.col(column)), r'\d+', '') == F.lit(''),
            F.lit('')
        ).otherwise(F.lower(F.col(column)))

    if bert_model:
        return df\
            .replace(fix_typos_dict, subset=column)\
            .withColumn(column, F.regexp_replace(F.col(column), units_and_symbols_expr, ''))\
            .withColumn(column, F.trim(F.col(column)))\

    else:
        return df\
            .replace(fix_typos_dict, subset=column)\
            .withColumn(column, F.regexp_replace(F.col(column), units_and_symbols_expr, ''))\
            .withColumn(column, __keep_only_compound_numbers())\
            .withColumn(column, F.trim(F.col(column)))\
            .where(F.length(F.col(column)) > F.lit(1))\
            .where(~F.col(column).isin(nltk.corpus.stopwords.words('english')))

A parte que mais importa

In [ ]:
if __name__ == '__main__':
    # Baixando conjuntos relevantes de palavras.
    nltk.download('wordnet', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    nltk.download('stopwords', quiet=True)

    MATCHED_SYNONYMS_PATH = './data/matched_synonyms/'
    CLEAN_PAPERS_PATH = './data/clean_results/'
    SYNONYM_ENTITIES = [x.lower() for x in ['Drug', 'Clinical_Drug', 'Pharmacologic_Substance']]

    # Cria a sessão do pyspark.
    ss()

    # janelas de agregação, definem critérios para agupamento de linhas do Spark Dataframe:
    w1 = Window.partitionBy(F.col('summary')).orderBy(F.col('filename'))
    w2 = Window.partitionBy(F.col('filename'), F.col('id')).orderBy(F.col('pos'))

    print('Preprocessing text for Word2Vec models')

    print('Replace synonyms: ' + str(True) + '\n')

    #####################################################################
    # PASSO 1
    # se for desejado substituir os compostos/drogas a partir de dados do PubChem:
        # cria a tabela de sinonimos. A primeira coluna contém o sinônimo do composto e a segunda coluna contém o nome (título) do composto ao qual aquele sinônimo se refere.
        # a coluna com os nomes dos sinônimos é transformada (sofre processamento), enquanto o título é apenas transformado em letras minúsculas
        # o grau da tabela original não é alterado. Ou seja, mantém-se a proporção 1 linha = 1 sinônimo

    # se o processamento do texto estiver sendo feito para treinamento de futuros modelos Word2Vec:
        # também é realizada a leitura do arquivo de texto que contém as palavras mais comuns do inglês. Esse arquivo é transformado em um DataFrame, removendo-se aquelas palavras selecionadas para o processo de validação
        # o DataFrame de palavras em inglês será usado para remover tais palavras do texto, antes do treinamento dos modelos.
    #####################################################################

    # se for ser realizada a normalização de sinônimos de compostos/drogas, é necessário criar seus Dataframes (incluindo o Dataframe de NER):
    synonyms = read_csv_table_files('./pubchem/synonyms/')
    synonyms = synonyms\
                .filter(F.col('cid') != "122172881")\
                .filter(F.col('cid') != "11104792")

    titles = read_csv_table_files('./pubchem/titles.csv', sep='|')
    titles = titles\
            .filter(F.col('cid') != "122172881")\
            .filter(F.col('cid') != "11104792")

    ner_df = read_csv_table_files('./ner/')\
            .where(F.col('entity').isin(SYNONYM_ENTITIES))

    print('ner_df:')
    ner_df.show(truncate=False)


    # se a normalização de sinônimos for ser realizada para futuro treinamento de modelos Word2vec, o Dataframe de sinônimos deve ser unido (join) ao Dataframe de palavras comuns do inglês,
    # pois elas serão removidas do texto:
    synonyms = synonyms\
            .withColumn('synonym', F.regexp_replace(F.lower(F.col('synonym')), r'\s+', ''))\
            .groupby('synonym')\
            .agg(F.min('cid').alias('cid'))\
            .join(titles, 'cid')\
            .withColumn('synonym_title', F.regexp_replace(F.lower(F.col('title')), r'\s+', ''))\
            .select('synonym', 'synonym_title')

    # independentemente de qual o futuro modelo a ser treinado, se houver noralização de sinônimos, o Dataframe de sinônimos é unido com o NER,
    # para que haja a normalização apenas de palavras identificadas como drogas/compostos/fármacos:
    synonyms = synonyms\
                .filter(F.col('synonym_title') != 'methyl(9r,10s,11s,12r,19r)-11-acetyloxy-12-ethyl-4-[(13s,15r,17s)-17-ethyl-17-hydroxy-13-methoxycarbonyl-1,11-diazatetracyclo[13.3.1.04,12.05,10]nonadeca-4(12),5,7,9-tetraen-13-yl]-8-formyl-10-hydroxy-5-methoxy-8,16-diazapentacyclo[10.6.1.01,9.02,7.016,19]nonadeca-2,4,6,13-tetraene-10-carboxylate')\
                .filter(F.col('synonym_title') != 'methyl(1r,10s,11r,12r,19r)-11-acetyloxy-12-ethyl-4-[(13s,15r,17s)-17-ethyl-17-hydroxy-13-methoxycarbonyl-1,11-diazatetracyclo[13.3.1.04,12.05,10]nonadeca-4(12),5,7,9-tetraen-13-yl]-10-hydroxy-5-methoxy-8-methyl-8,16-diazapentacyclo[10.6.1.01,9.02,7.016,19]nonadeca-2,4,6,13-tetraene-10-carboxylate')

    synonyms = synonyms\
                .where(F.col('synonym') != F.col('synonym_title'))\
                .join(ner_df, F.col('synonym') == F.col('token'), 'inner')\
                .drop(*('token', 'entity'))

    #####################################################################
    # PASSO 2
    # cria o DataFrame de artigos limpos/processados. Cada linha dessa tabela equivale a um artigo.
    # a tabela tem três colunas: filename, id, summary
    #       "filename" é o nome do arquivo de onde o artigo foi retirado (results_aggregated), ou seja, é o ano de publicação do artigo.
    #       "id" é uma coluna serial, apenas para contagem/identificação
    #       "summary" é o próprio texto (título e/ou prefácio do artigo) limpo/processado.
    #####################################################################

    cleaned_documents = read_csv_table_files('../bert/results/', sep='|')
    print('Abstracts originais:')
    cleaned_documents.show(truncate=False)

    cleaned_documents = cleaned_documents\
                        .withColumn('summary', summary_column_preprocessing(F.col('summary'), bert_model=False))\
                        .select('id', 'filename', F.posexplode(F.split(F.col('summary'), ' ')).alias('pos', 'word'))

    print('Após summary_column_preprocessing:')
    cleaned_documents.show(truncate=False)

    cleaned_documents = words_preprocessing(cleaned_documents, bert_model=False)\
                        .withColumn('summary', F.collect_list('word').over(w2))\
                        .groupby('id', 'filename')\
                        .agg(
                            F.concat_ws(' ', F.max(F.col('summary'))).alias('summary')
                        )

    print('Após words_preprocessing:')
    cleaned_documents.show(truncate=False)

    #####################################################################
    # PASSO 3
    # se o texto estiver sendo processado para modelos BERT, NÃO é realizada a lemmatização dos verbos e advérbios
    # caso contrário, é realizada a lemmatização
    # em ambos os casos, o texto (summary) é tokenizado nos espaços em branco, formando uma linha do Dataframe para cada token
    # esse novo Dataframe - com os tokens - será utilizado para união (join) com o dataframe de sinônimos
    #####################################################################

    df = cleaned_documents\
        .select(
            'id',
            'filename',
            F.posexplode(F.split(F.col('summary'), ' ')).alias('pos', 'word')
        )

    print('Após primeiro posexplode:')
    df.show(truncate=False)

    df = df\
        .withColumnRenamed('word', 'n_word')

    df.show(n=60, truncate=False)

    df = df\
        .join(synonyms, F.col('synonym') == F.lower(F.col('n_word')), 'left')\
        .distinct()

    df.show(truncate=False)

    matched_synonyms = df\
        .where(F.col('synonym_title').isNotNull())\
        .select(F.col('synonym'), F.col('synonym_title'))\
        .distinct()\
        .where(F.col('synonym') != F.col('synonym_title'))

    df = df\
        .withColumn('word', F.coalesce(F.col('synonym_title'), F.col('n_word')))\
        .drop(*('synonym', 'synonym_title', 'n_word'))\
        .withColumn('summary', F.collect_list('word').over(w2))\
        .groupby('id', 'filename')\
        .agg(
            F.concat_ws(' ', F.max(F.col('summary'))).alias('summary')
        )

    print('Final - após possível normalização de sinônimos:')
    df = df.withColumn('id', F.monotonically_increasing_id())
    df.show(n=60, truncate=False)

    print('Escrevendo csv')
    to_csv(df, target_folder=CLEAN_PAPERS_PATH)

    to_csv(matched_synonyms, target_folder=MATCHED_SYNONYMS_PATH)

    df.printSchema()
    print('END!')

# 4. Treinamento do modelo

Com os dados limpos e pré-processados, o modelo Word2Vec pode ser treinado.

In [ ]:
import os, re, sys, shutil, itertools

import gensim
from gensim.utils import RULE_KEEP, RULE_DEFAULT
from gensim.models import Word2Vec, FastText
from pathlib import Path
from os import listdir
import pandas as pd
import numpy as np

In [ ]:
def list_from_txt(file_path):
    '''Creates a list of itens based on a .txt file, each line becomes an item.

    Args:
      file_path: the path where the .txt file was created.
    '''

    strings_list = []
    with open (file_path, 'rt', encoding='utf-8') as file:
        for line in file:
            strings_list.append(line.rstrip('\n'))
    return strings_list

def clear_folder(dirpath):
    """ Clears all files from a folder, without deleting the folder.

        dirpath: the path of the folder.
    """

    for filename in os.listdir(dirpath):
        filepath = os.path.join(dirpath, filename)
        try:
            shutil.rmtree(filepath)
        except OSError:
            os.remove(filepath)

def get_target_compounds():
    return sorted(['cytarabine', 'daunorubicin', 'azacitidine', 'midostaurin', 'gemtuzumab-ozogamicin', 'vyxeos', 'ivosidenib', 'venetoclax', 'enasidenib', 'gilteritinib', 'glasdegib', 'arsenictrioxide', 'cyclophosphamide', 'dexamethasone', 'idarubicin', 'mitoxantrone', 'pemigatinib', 'prednisone', 'rituximab', 'thioguanine', 'vincristine'])

def keep_target_compounds(word, countm, min_count):
    if word in get_target_compounds() + ['aml']:
        return gensim.utils.RULE_KEEP

    else:
        return gensim.utils.RULE_DEFAULT

In [ ]:
if __name__ == '__main__':
    print('Starting script')

    # CONSTANTS:
    MODEL_TYPE = 'ft' # 'w2v' for Word2Vec or 'ft' for FastText
    if MODEL_TYPE == 'w2v':
        os.makedirs('./models_yoy_combination15/', exist_ok=True)
        os.makedirs('./models_yoy_combination2/', exist_ok=True)

        parameters_combination = [[100, 0.0025, 10], [200, 0.025, 15]]

    else:
        os.makedirs('../fasttext/models_yoy_combination16/', exist_ok=True)

        parameters_combination = [[300, 0.0025, 5]]

    # leitura do arquivo .csv em um DataFrame:
    print('Reading DataFrame of papers')
    df = pd.read_csv('./pubchem/results_pandas.csv', escapechar='\\')

    # todos os anos de publicação (sem repetição) presentes no arquivo .csv:
    years = sorted(df.filename.unique().tolist())
    first_year = years[0]

    # intervalos ou "janelas" de tempo, a partir do primeiro artigo publicado, até o último. Exemplo:
    # supondo que o primeiro artigo coleto foi publicado em 1921, e que, depois deste, mais artigos foram publicados nos anos seguintes, temos:
    # [[1921], [1921, 1922], [1921, 1922, 1923], [1921, 1922, 1923, 1924], [1921, 1922, 1923, 1924, 1925], .......]
    ranges = [years[:i+1] for i in range(len(years))]
    ranges = [x for x in ranges if 1980 in x]
    ranges = [x for x in ranges if x[-1] == 1980]

    for r in ranges:
        print('training model from {} to {}'.format(r[0], r[-1]))
        abstracts = df[df.filename.isin(r)]['summary'].to_list()
        print('number of abstracts: {}\n'.format(len(abstracts)))
        abstracts = [x.split() for x in abstracts]

        # train model
        if MODEL_TYPE == 'w2v':            
            model_comb15 = Word2Vec(
                            # constant parameters:
                            sentences=abstracts,
                            sorted_vocab=True,
                            min_count=5,
                            sg=1,
                            hs=0,
                            iter=15,
                            trim_rule=keep_target_compounds,
                            # variable parameters:
                            size=parameters_combination[1][0],
                            alpha=parameters_combination[1][1],
                            negative=parameters_combination[1][2]
                        )
            model_comb15.save('./models_yoy_combination15/model_{}_{}.model'.format(first_year, r[-1]))

        else:
            model = FastText(
                    # constant parameters:
                    sentences=abstracts,
                    sorted_vocab=True,
                    min_count=5,
                    sg=1,
                    hs=0,
                    iter=15,
                    trim_rule=keep_target_compounds,
                    # variable parameters:
                    size=parameters_combination[0][0],
                    alpha=parameters_combination[0][1],
                    negative=parameters_combination[0][2])
            model.save('/fastdata/ac4mvvb/fasttext/models_yoy_combination16/model_{}_{}.model'.format(first_year, r[-1]))

    print('END!')